In [3]:
from gym import Env
from gym.spaces import Discrete, Box, MultiDiscrete
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
import numpy as np

C:\Users\gabri\anaconda3\lib\site-packages\tensorflow\core\framework\tensor_shape_pb2.py:18: DeprecationWarning: Call to deprecated create function FileDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  DESCRIPTOR = _descriptor.FileDescriptor(
C:\Users\gabri\anaconda3\lib\site-packages\tensorflow\core\framework\tensor_shape_pb2.py:36: DeprecationWarning: Call to deprecated create function FieldDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  _descriptor.FieldDescriptor(
C:\Users\gabri\anaconda3\lib\site-packages\tensorflow\core\framework\tensor_shape_pb2.py:29: DeprecationWarning: Call to deprecated create function Descriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  _TENSORSHAPEPROTO_

In [4]:
class Policy:
    def __init__(self):
        self.alpha = 0.01
        self.policy = keras.models.Sequential([
            keras.layers.Dense(25, activation="relu", input_shape=[25]),
            keras.layers.Dense(10, activation="relu"),
            keras.layers.Dense(5, activation="softmax"),
        ])
        self.policy.compile(optimizer=Adam(learning_rate=self.alpha))
        
    def __call__(self, state):
        pi = self.policy(state)
        return pi

In [5]:
# Criando conta para complementar o ambiente de treinamento
class Account:
    def __init__(self, accountId=0, balance=random.randint(-100, 100), customerId=0, organizationId=0, creditCards=0, bill=random.randint(-100, 100)) -> None:
        self.accountId = accountId
        self.balance = balance
        self.customerId = customerId
        self.organizationId = organizationId
        self.creditCards = creditCards
        self.bill = bill
        self.log = 'Detalhes das contas do cliente ' + self.customerId + '\n' + '----------------------------------------\n'

    def showAllSituation(self) -> str:
        return f"Conta: {self.accountId} | Saldo: {self.balance} | Cliente: {self.customerId} | Organização: {self.organizationId} | Cartões de crédito: {self.creditCards} | Fatura: {self.bill}"
        
    def getAccountId(self) -> str:
        return self.accountId
    
    def getBalance(self) -> float:
        return self.balance
    
    def getCustomerId(self) -> str:
        return self.customerId
    
    def getOrganizationId(self) -> str:
        return self.organizationId
    
    def toWithdraw(self, amount: float) -> float:
        self.log(f"Saque de {amount} na conta {self.accountId} em {datetime.now()}\n")
        self.balance -= amount
        return amount
    
    def toDeposit(self, amount: float) -> None:
        #self.log += f"Depósito de {amount} na conta {self.accountId} em {datetime.now()}\n"
        self.balance += amount
    
    def toTransfer(self, amount: float, account) -> None:
        if account in self.allAccounts:
            self.log += f"Transferência de {amount} da conta {self.accountId} para a conta {account.getAccountId()} em {datetime.now()} (Remanejamento entre suas contas)\n"
            self.balance -= amount
            account.balance += amount
        else:
            self.log += f"Transferência de {amount} da conta {self.accountId} para a conta {account.getAccountId()} em {datetime.now()} (Remanejamento entre contas de terceiros)\n"
            self.balance -= amount
            account.balance += amount

In [10]:
'''
Vamos criar um ambiente de treinamento para o agente (Brain Balance Control)
'''
class BBCEnv(Env):
    def __init__(self, time_limit=5475, n_accounts=4, salary=random.randint(1000, 10000)):
        self.time = 0
        self.myAccounts = [Account() for i in range(n_accounts)]
        self.action_space = Discrete(4)
        self.obs_shape = (7,)
        self.observation_space = gym.spaces.utils.flatten_space([1]*7)
        self.time_limit = time_limit
        self.salary = salary
        self.faturas = 1000 + random.randint(0, 10000)
        self.gamma = 0.99
        self.investimentos = 0
        self.j = 0.08 # Cheque especial

    def get_obs(self):
        # Accounts 
        saldos = [account.getBalance() for account in self.myAccounts]
        salary = self.salary
        if self.time % 30 == 0:
            fatura = (self.fatura + random.randint(-1000, 1000)) * self.gamma
            self.fatura = fatura

        obs = saldos + [salary] + [self.fatura] + self.investimentos
        return np.array(obs)

    # Jump
    def apply_action(self, action):
        if action = 0:
            self.remanejamento()
        elif action = 1:
            self.emprestimo_consignado()
        elif action = 2:
            self.investir()
        elif action = 3:
            self.pagar_com_investimento()

    def step(self, action):
        # Get obs
        obs = self.get_obs()

        # Coleta saldo negativo
        rew = 0
        contas_negativas = []
        for i in range(len(self.myAccounts)):
            conta = self.myAccounts[i]
            if conta.getBalance() < 0:
                if self.time % 30 == 0:
                    self.myAccounts[i].balance = self.myAccounts[i].getBalance() * (1 + self.j)
                    contas_negativas.append(self.myAccounts[i].getBalance())
                else:
                    contas_negativas.append(conta.getBalance())
                rew += contas_negativas[-1]
        if self.time % 30 == 0:
            for i in range(len(self.myAccounts)):
                self.myAccounts[i].toDeposit(self.salary)
        
        
        
        
            
        
        reward = 2 ** ngbs.count(2)
        # Get info
        info = {}

        # Check if done
        if self.time >= self.time_limit:
            done = True
            return obs, reward, done, info
        else: 
            done = False

        # apply action
        self.apply_action(action)
        self.time += 1
        return obs.reshape(1, -1), reward, done, info

    def render(self):
        print('-' * self.L)
        print(self.array)
        print('-' * self.L)

    def reset(self):
        self.pos = (self.L // 2, self.L // 2)
        self.action_space = Discrete(4)
        self.observation_space = gym.spaces.utils.flatten_space(MultiDiscrete(np.array([[1 for i in range(5)] for j in range(5)]))) # MultiDiscrete(np.array([[3 for i in range(5)] for j in range(5)]))
        self.types = Discrete(3)
        self.array = np.array([[Discrete(2).sample() * 2 for i in range(self.L)] for j in range(self.L)])
        # Put one predator
        self.array[self.pos[0], self.pos[1]] = 1               
        self.time = 0
        return self.get_obs().reshape(1, -1)